In [ ]:
# 遺漏值的處理方式



# 正常步驟 , 分成三個部分 : 

# 1. 辨識出四種遺漏值 , NaN , "" , " " , None

# 2. 將遺漏值轉換成 NaN

# 3. 用 pandas , numpy 或 python 內建函數來移除 NaN




In [ ]:
# 1. 辨識遺漏值


# a. NaN : 

# 型別 : 屬於數值型的類別，常常是計算產生，例如無窮大，或者是數值空格沒填資料，float("NaN") , numpy.nan , pandas.na 都算是這種

# 判斷 : NaN 因此不能用 == 來辨識，要用各個模組的函數，像是 math.isnan() , numpy.isnan() , dataframe.isna() 之類


# b. "" : 

# 型別 : 是空字串，但是還是算一種資料，屬於 string 

# 判斷 : 可以用 == "" 來偵測


# c. " " : 

# 型別 : 是空白的字串，屬於 string

# 判斷 : 因為可能有多個空白，所以要搭配 strip() == "" 把空白拿掉確認沒有東西來辨識


# d. None : 

# 型別 : 是 Python 的一種型別，就是 None 型別

# 判斷 : 可以直接用 is None 來偵測 , None 在 Pandas 也算是 NaN , 也可以用 isna() 偵測

# 雖然 None 可以用 isna() 偵測 , 但是數值和字串欄位可能都有 None , 都要各自做 None 的偵測



In [ ]:
import pandas



# 2. 轉換成 NaN



# 因為多數模組有提供辨識 NaN 並移除的功能，因此可以轉換完成 NaN 辨識並移除

# 轉換 NaN 主要是用 replace 來進行



# a. 下面用 pandas dataframe 舉例，有五種類型的遺漏值

data_column = ["日期","名稱","收盤價"]

data_row = [["20250801","aaa",120],["20250802","aaa",float("Nan")],["20250803","aaa", ],["20250804","   ",150],[None,"aaa",160]]

dataframe = pandas.DataFrame(data_row,columns=data_column)

print(dataframe)


# b. 找出這些遺漏值並轉成 NaN

# 要把不同資料類型的欄位區分出來做，數值歸數值，字串歸字串，不然 pandas.isna() / numpy.isnan() 會報錯

# b-1. 用 select_dtypes() 來挑選出文字欄位，會回傳一個挑選欄位的 dataframe

select_dataframe = dataframe.select_dtypes(include="object")

print(select_dataframe)

# b-2. 用 columns 迴圈來做 strip() , 因為 str.strip() 只能給 series 用 , dataframe 不能用

for col in select_dataframe.columns : 

    dataframe[col] = dataframe[col].str.strip()

print(dataframe)

# 第二種寫法 : 用 applymap() , 就是 apply() 逐行 + map() 逐個元素 處理，所以效能比較不好

    # 下面就是前面元素被迭帶代入 strip() , 像 map(strip()) 一樣 
    
    # 後面判斷是 str 才做 strip(x) 並傳出，不是就維持原來 x 傳出

    # dataframe = dataframe.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # isinstance() 就是 in [...] 的函數版本，不過特別是看是不是哪個型別，例如 str , float ...

    # 如果 isin / isinstance = is [...] 或 == [...] , 那 isna / isnan 就是 is nan 或 == nan 

    # isna / isnan / isin / isinstance 都是回傳布林值的函數

# 第三種寫法 : 用 apply() 直接逐行處理，效能比較好

    # 不過就不用 for 迴圈，直接給 iterable 就可以，lambda 基本上就是迴圈，map() 那些也都是

    # list comprehension = for 迴圈 = lambda = map() , apply() , frompyfunc() 

    # lambda 本質上應該是迴圈內增加篩選條件用 , 是指迴圈內的元素 , 而不是他在產生迴圈 , 
    
    # 實際上還是要靠 map() , filterfalse() 這類函數本身有元素逐項讀取的迴圈功能才能迴圈 , 
    
    # 只是因為迴圈函數常常配合 lambda 來代表元素和(前)處理元素 , 所以才會在迴圈函數常看到 lambda

    # lambda 通常是用在一個函數不夠用 , 需要兩個以上 ... and ... 邏輯運算或 ... if ... 的情況，

    # 下面一個函數情況的 apply(lambda col : col.str.strip()) 就可以省略為 apply(str.strip)

    # 或者像 filterfalse(lambda x : math.isnan(x) , a) 可以省略為 filterfalse(math.isnan , a)

    # select_columns = select_dataframe.columns

    # dataframe[select_columns] = dataframe[select_columns].apply(lambda col : col.str.strip())

    # 或下面接近 map() 的寫法，省略裡面呼叫函數的括號，apply() 代入的變數就是前面的 iterable 的元素

    # dataframe[select_columns] = dataframe[select_columns].apply(str.strip)

# b-3. 接下來就用 replace 把 strip 完的 dataframe 的所有文字遺漏值轉換成 NaN

# 記得要加 inplace = True

dataframe.replace(["",None],float("NaN"),inplace=True)

print(dataframe)



# 3. 移除 NaN

# 轉換成 NaN 之後用模組的函數來移除，例如 pandas.dropna()

# 最後用 dropna() 把遺漏值行去掉

# 記得要加 inplace = True

dataframe.dropna(inplace=True)

print(dataframe)



# 4. 找出 NaN 

# 只能用布林陣列的方法 , 簡單來說就是用 isin([ ... , ... ]) 還有 isna() 本身為 True 來找出異常列，

# 只是因為遺漏值有分字串和數值，而找空白的 strip() 不能用在數值欄位，因此要分字串和數值欄位處理

# 不然其實 isna() 就已經可以把 NaN 資料列篩選出來，pandas 的 isna() 是可以用在字串欄位的，不像 numpy 的 isnan() 不能

dataframe_2 = pandas.DataFrame(data_row,columns=data_column)

# isna() 可以直接對 dataframe 使用，產生布林陣列

# 然後用 any(axis=1) 選出只要任一欄有 NaN 的資料列就為 True，axis = 1 就是往欄方向看

# isna().all(axis=1) 則是選出全部欄位都是 NaN 的為 True

# notna().all(axis=1) 則是選出全部欄位都要是沒有 NaN 的資列列為 True

NaN_rows = dataframe_2[dataframe_2.isna().any(axis=1)]

print(NaN_rows)

# 字串欄也是一樣的方法

Empty_rows = dataframe_2.select_dtypes(include="object")

# 因為是慢慢累加 True 的資料列，所以要用 a = a + b

Empty_columns = Empty_rows.columns

Empty_rows = pandas.DataFrame()

for col in Empty_columns : 

    Empty_rows = pandas.concat([Empty_rows,dataframe_2[dataframe_2[col].str.strip().isin([None,""])]])

print(Empty_rows)

# 最後會發現有重複的列 , 因為當初做 None 的篩選的時候 , 數值和字串有一起做 , 才接下來做字串的

# 沒有把數值和字串列分開 , 所以會有重複 , 可以分開做 , 就不會有重複的列




         日期   名稱    收盤價
0  20250801  aaa  120.0
1  20250802  aaa    NaN
2  20250803  aaa    NaN
3  20250804       150.0
4      None  aaa  160.0
         日期   名稱
0  20250801  aaa
1  20250802  aaa
2  20250803  aaa
3  20250804     
4      None  aaa
         日期   名稱    收盤價
0  20250801  aaa  120.0
1  20250802  aaa    NaN
2  20250803  aaa    NaN
3  20250804       150.0
4      None  aaa  160.0
         日期   名稱    收盤價
0  20250801  aaa  120.0
1  20250802  aaa    NaN
2  20250803  aaa    NaN
3  20250804  NaN  150.0
4       NaN  aaa  160.0
         日期   名稱    收盤價
0  20250801  aaa  120.0
         日期   名稱    收盤價
1  20250802  aaa    NaN
2  20250803  aaa    NaN
4      None  aaa  160.0
         日期   名稱    收盤價
4      None  aaa  160.0
3  20250804       150.0


In [ ]:
import pandas
import numpy



# 4. 其他方式 1 : 布林陣列

# pandas / numpy : isna() , isnan() 形成布林陣列來篩選移除



# a. pandas 的作法 : 


data_column = ["日期","名稱","收盤價"]

data_row = [["20250801","aaa",120],["20250802","aaa",float("Nan")],["20250803","aaa", ],["20250804","   ",150],[None,"aaa",160]]

dataframe = pandas.DataFrame(data_row,columns=data_column)

# 複製 dataframe 可以用 copy()

dataframe_2 = dataframe.copy()

print(dataframe_2)


# a-1. 先區分出字串和數字欄位，不然用字串用 isna() 會報錯

dataframe_string = dataframe.select_dtypes(include="object")

dataframe_number = dataframe.select_dtypes(include="number")

print(dataframe_string)

print(dataframe_number)


# a-2. 再各自用不同函數做布林陣列篩選

# 分多次去掉不要的資料列即可，先做字串欄位篩選，再做數字欄位篩選

# 字串欄位就用 ~ 加上 isin()

select_string = dataframe_string.columns

for col in select_string : 

    dataframe = dataframe[~dataframe[col].str.strip().isin(["",None])]

# 數字欄位就用 isna() 或 notna()

select_number = dataframe_number.columns

for col in select_number : 

    dataframe = dataframe[dataframe[col].notna()]

# 這邊 lambda 寫法會出錯 

# dataframe = dataframe[~dataframe[select_string].apply(lambda col : col.str.strip().isin(["",None]))]

# dataframe = dataframe[~dataframe[select_number].apply(lambda col : col.isna())]

print(dataframe)


# a-3. 如果要找出 NaN 的資料列，就把上面條件反過來的資料列整合即可

# 不過和上面逐漸去掉是反過來的，因為是有找到新的列就加上去，所以要 concat()

# 另外要加上宣告 dataframe = pandas.Dataframe() , 因為是類似 a = a + b , 也要寫 a = 0 , 如果沒有那段，記憶體會一直累加導致錯誤

dataframe_3 = pandas.DataFrame()

for col in select_string : 

    dataframe_3 = pandas.concat([dataframe_3,dataframe_2[dataframe_2[col].str.strip().isin(["",None])]])

print(dataframe_3)

dataframe_4 = pandas.DataFrame()

for col in select_number : 

    dataframe_4 = pandas.concat([dataframe_4,dataframe_2[dataframe_2[col].isna()]])

dataframe_combine = pandas.concat([dataframe_3,dataframe_4])

print(dataframe_combine)



# b. numpy 的作法

# b-1. numpy 基本上是處理數值為主的模組 , 因此提供的去除遺漏值方法只有數值的 isnan()

# b-2. numpy 也沒辦法處理有空字串的情況，如果有空格沒有放元素會錯誤 , 像是 ["20250801","aaa", ] 這樣

array = numpy.array([[1,numpy.nan,3],[4,5,numpy.nan],[numpy.nan,8,9]])

# b-3. 但因為 numpy 本質是 list , 也是可以放數值以外的變數當作元素

array_2 = numpy.array([["a",2,3],["b",5,6],["c",8,9]])

print(array_2)

# 但是只要是混和型 array , 全部元素都會自動轉型成 u21 或 u32 的混和型文字型別 , 用 dtype 屬性可以檢視

# 或者是上面 array_2 檢視結果就可以發現都是字串

print(array_2[0:1].dtype)

# 因此即便 numpy 有 dtype , 還是不能代替沒有 select_dtypes() 的問題，因為都轉型成混和字串類別

# 可以看下面例子，執行後就會發現因為都是字串，numeric_array 會是空的，value 可以發現都是錯

# 目前只能用一個一個元素去檢驗的方式來判斷 , 用 isinstance() + all()

# numeric_array = [index for index in range(array_2.shape[1]) if all([isinstance(item,(int,float)) for item in array_2[:,index]])]

# print(numeric_array)

# for index in range(array_2.shape[1]) : 

#     value = [type(item) for item in array_2[:,index]]

#     print(value)

# 結論 : 還是用 pandas 來做遺漏值排除會比較好，或者用下面的 list comprehension 一個一個元素看

#        numpy 基本上還是數值型的陣列，主要是做為高效率運算用



         日期   名稱    收盤價
0  20250801  aaa  120.0
1  20250802  aaa    NaN
2  20250803  aaa    NaN
3  20250804       150.0
4      None  aaa  160.0
         日期   名稱
0  20250801  aaa
1  20250802  aaa
2  20250803  aaa
3  20250804     
4      None  aaa
     收盤價
0  120.0
1    NaN
2    NaN
3  150.0
4  160.0
         日期   名稱    收盤價
0  20250801  aaa  120.0
         日期   名稱    收盤價
4      None  aaa  160.0
3  20250804       150.0
         日期   名稱    收盤價
4      None  aaa  160.0
3  20250804       150.0
1  20250802  aaa    NaN
2  20250803  aaa    NaN
[['a' '2' '3']
 ['b' '5' '6']
 ['c' '8' '9']]
<U21
[<class 'numpy.str_'>, <class 'numpy.str_'>, <class 'numpy.str_'>]
[<class 'numpy.str_'>, <class 'numpy.str_'>, <class 'numpy.str_'>]
[<class 'numpy.str_'>, <class 'numpy.str_'>, <class 'numpy.str_'>]


In [22]:
import numpy
import pandas
import math
from itertools import filterfalse



# 5. 其他方式 2 : python 原生方式

# List comprehension + if : 直接一個一個元素辨識並挑選不是遺漏值的元素

# 三個分辨方式

# a. "" , " " : 字串 , 所以要 isinstance(x,str) and strip(x) == True 才做

# b. Nan : 數值 , 所以要 isinstance(x,float) and math.isnan(x) == True 才做

# c. None : None 類別 , 所以要 is None 才做


# 一維的 list 用 list comprehension 就可以了

# 一維的 numpy 陣列可以用 for 迴圈一個一個讀取元素 , 但是因為不能混和放字串或數字 , 否則全部轉文字 , 因此只有舉屬值的例子

# 寫法一 : 最基本寫法 , 多次的 list comprehension , 也可以用 for 迴圈代替

NaN_list = [1,""," ",float("NaN"),None]

NaN_list = [x for x in NaN_list if not (isinstance(x,str) and x.strip() == "")]

NaN_list = [x for x in NaN_list if not (isinstance(x,float) and math.isnan(x))]

NaN_list = [x for x in NaN_list if not (x == None)]

print(NaN_list)

NaN_list_2 = [1,""," ",float("NaN"),None]

New_NaN_list = []

for x in NaN_list : 

    if isinstance(x,str) and x.strip() == "" : 

        continue

    if isinstance(x,float) and math.isnan(x) : 

        continue

    if x == None : 

        continue

    New_NaN_list.append(x)

print(New_NaN_list)

# 寫法二 : 用多次 filterfalse 代替 list comprehension , 最後要記得轉 list 才能看到結果

NaN_list_3 = [1,""," ",float("NaN"),None]

NaN_list_3 = filterfalse(lambda x : isinstance(x,str) and x.strip()=="",NaN_list_3)

NaN_list_3 = filterfalse(lambda x : isinstance(x,float) and math.isnan(x),NaN_list_3)

NaN_list_3 = filterfalse(lambda x : x == None,NaN_list_3)

NaN_list_4 = list(NaN_list_3)

print(NaN_list_4)


# 寫法三 : 直接用 any() 把所有條件合併

NaN_list_5 = [1,""," ",float("NaN"),None]

NaN_list_5 = [x for x in NaN_list_4 if not any([(isinstance(x,str) and x.strip() == ""),(isinstance(x,float) and math.isnan(x)),(x == None)])]

print(NaN_list_5)



# array = numpy.array([1,float("NaN"),3])







# 下面做適用二維陣列的寫法 : 利用 pandas dataframe 和 numpy array 基本上就是 list , 第一個元素就是一列的 list 方式來加上多次 list comprehension 來做


# dataframe 

data_column = ["日期","名稱","收盤價"]

data_row = [["20250801","aaa",120],["20250802","aaa",float("Nan")],["20250803","aaa", ],["20250804","   ",150],[None,"aaa",160]]

dataframe = pandas.DataFrame(data_row,columns=data_column)

# for i in dataframe.shape[0] : 

#     dataframe[i] = [x for x in dataframe[i] if ]


[1]
[1]
[1]
[1]
